Done
=====
1. Generate dataset instead of reading it in.
1. Re-evaluate toolbox of methods after inlining. Consider doing more detailed analysis of why it changes in performance.
    1. Note that with inlining turned on, the differences are quite small, however, some of the optimizations have larger code size improvements.
    2. Detail what methods were used and how they impacted performance. Compiler explorer will help.
        1. Binary search.
        2. Interpolation search.
3. Measure performance of each improvement.
3. Measure which dataset attributes predict performance of interpolate-linear search.
4. Measure impact of threading on performance across dataset sizes.
3. Measure performance of best recursive and linear search versions of interpolation and binary search across dataset sizes and seeds.
1. Write report on toolbox of methods. Only measure full selection since we don't know anything about variance of searches.
    1. Show performance of each improvement, and detail why the optimization improves performance.
    3. Show which dataset attributes predict performance of interpolate-linear search.
    4. Show impact of threading on performance across dataset sizes. Across sizes is important because cache thrashing may affect sizes differently, so y axis should be degradation, not time.
    3. Show performance of best recursive and linear search versions of interpolation and binary search across dataset sizes and seeds. Consider skipping full multi-threaded measurements for large sizes.
8. Increase granularity of b-sz measurements and some re-labeling.

TODO
=====
1. Fill out sections on threads, different sizes.
3. Re-label graphs.
4. Edit section on optimizations.
2. Add an evaluation for i-lin-simd
    
Failed Optimizations
-----------------------
1. Immediate return and break were worse. Likely because they no longer generated cmov instr. Important and non-intuitive.
1. Divide into several intervals, count the number of comparisons passed, and recurse on remaining sub-interval.
2. Use array left, mid, right. Choose left right based on x which is determined by the comparison. Already have branchless.
3. Interpolation search followed by exponential search.
4. Use derivative of the slope to reduce to multiplication.
5. Alternate interpolation with linear search.
6. Store linear splines on the array and interpolate on the containing spline.
7. Do two interpolations and linear search.
8. Use interger division for interpolation.
9. Test for equality first.
10. Guard linear search by indexes.
11. Use new shift on every iteration.

Subsets Study
--------
1. Break measurements into subsets and measure differences across sizes. Find minimal subset size. May vary across array sizes.
2. Vary seed for subset choice as well as dataset generation.
3. How similar are searches for different subsets? Subset variation predicts point query variation.
4. How much impact does branch prediction make? Subset search in sub-cycles.

Large Array Improvements
-------------
1. Improve linear search with gather + skip strategies.
3. Plot against size, selection, threads on larger arrays.
8. Improve search by incorporating binary search on the error.
9. How many datasets needed to be representative?

Results
=====
* Why is this important? Search is well-studied, but interpolation is neglected. In-memory workloads are important, and increasingly so with data visualization platforms becoming interactive.
* Consider discussing the case where the key isn't in the array.

Key Contributions
--------------------
1. List of optimizations for binary search.
2. List of optimizations for interpolation search.
3. Comparison of interpolation search and binary search.
    1. Full selection across sizes.
    2. Concurrent search / multi-core minimally degrades performance.
5. Relationship between mathematical properties of data and performance of interpolation search.
    * When to use interpolation search?

Notes
====
* It might also work to require the quotient be no less then one, then do a lookup on the whole division operation by scaling down dividend and the divisor by the log of the divisor and looking up the output.
* Another idea for the recursive case is to simply build a table of slopes, and to do a lookup of the appropriate slope for the given interval simply using the midpoint.
* Since the recursive interpolation is a win in the large array case, perhaps the number of iterations in the linear search can be parameterized by the size of the array. Indeed, you could stack up a bunch of interpolations and do guard checks and a size check. If you hit the guard check or the size check, then jump to the outro linear search. On small arrays, you add the additional cost of the size check, so some thought should be given to how to avoid that. Maybe a single interpolation should be a special case that always does the linear search.
* Using guards was strictly worse than doing a single interpolation for the small array case, but is it possible that it might help the recursive case? Setting code bloat aside, maybe do a single interpolation followed by a short linear search, and then do a recursive interpolation search with guards. Maybe even just do a recursive interpolation search with linear search without explicitly special casing the first one.
* SIMD vs Unroll is an unstable choice.
* Can use maximum distance information to choose a linear search?
* while loop does better until the unrolling. Maybe do runtime unrolling of while loop manually?
* I don't think it's needed to do pinning because very little degredation in performance has been measured and pinning should make the benchmarks no worse.
* To measure the effect of branch prediction, we discussed altering the subsets search in such a way that each subset would be searched using sub-cycles of length 2k. For k = 2, this would be like (A, B, A, B, C, D, C, D, ...). For simplicity, simply starting at k=1, which simply repeats each query twice.
* subsets to get amortization, more subsets to learn about variance of searches. Needed for validity of selection results.
* Consider a view where the workset is held constant, but the dataset is increased.
* Important to specify that the bars in the performance graphs describe the performance of a single algorithm, with a low amount of error on different datasets.
* For designing how to do the subsets, if the order of subset searches is defined, then, within a thread, it's easy to reconstitute which subset a particular measurement belonged to. Since we're focusing on outputing all samples, don't need to worry about sorting output.

In [ ]:
from pandas import read_csv,DataFrame,Series,concat
from os import popen
from subprocess import call,check_output
from math import sqrt,log2, floor
from matplotlib.pyplot import show, plot, table, subplots, title, legend, gca
from scipy.stats import linregress
from sklearn.metrics import r2_score,mean_absolute_error
from heapq import heappush,heappop
from numpy import array,fromiter,int64,histogram
from IPython.display import display
from itertools import accumulate
from collections import Counter
import random as rnd
n_files = n_samples = 11

max_n_thds = 4
szs = [10**x for x in range(3,6 +1)]

rnd.seed(42)
seeds = [rnd.randint(1, 2**63) for _ in range(n_files)]
# this is only true for 100% selection
runs = [5000] + [10]*8
def mode(l): return Counter(l).most_common(1)[0][0]
def matrix_norm_vec(m, v): return (v / m.T).T
def ip_range(s, q=.25): return (s.quantile(1-q) - s.quantile(q))/s.median()
def quantile_p(s, q): return abs((s.quantile(q) - s.median())/s.median())

b_lr_algs = 'b-lr b-lr-cond  b-lr-over b-lr-noeq b-lr-for b-lr-noeq-for b-lr-lin'
b_sz_algs = 'b-sz b-sz-cond b-sz-noeq b-sz-for b-sz-noeq-for b-sz-pow b-sz-lin'
i_algs = 'i i-precompute i-lut i-seq-fp i-seq i-seq-simd'

algs = ' '.join([b_lr_algs, b_sz_algs, i_algs])
sz_algs = 'i-guard i-seq b-sz-lin'

# for margin of error e_m, critical value z, uncertainty sigma, mean mu
# n = ((z*sigma)*2/(e_m*mu))^2

Toolbox of Methods
=========
1000 elements, 100% selection, all algorithms, many seeds.

Structure discussion around a compilation of effectiveness of methods. Make a table of the toolbox of methods. Show impact of each method separately. A --> B --> C

The overflow math optimization only makes sense in the context of left + right. I'm not sure that L + R is faster or slower than L + N, but I have found more optimizations for L + N. Less state changes with L + R, so it may have less work, but L + N may allow for more work to be done independently.

Individually, independent calculations and the linear search appear to do worse, however, together, they seem to improve.

<table>
<tr><th>Method</th><th>LR Impact</th><th>Size Impact</th><th>Interpolation Impact</th></tr>
<tr><td>Overflow Math.</td><td>Increased ILP.</td><td>N/A</td><td>N/A</td></tr>
<tr><td>Don't test equality.</td><td>Remove 1 out of 3 conditional moves and save an increment.</td><td>Didn't try.</td><td>Measured to be worse.</td></tr>
<tr><td>Reduce size to power of 2.</td><td>Didn't try.</td><td>Save on subtraction in core loop.</td><td>Didn't try, not likely to stay power of 2.</td></tr>
<tr><td>Precompute logarithm and use for loop.</td><td>Run-time loop unrolling.</td><td>Run-time loop unrolling. Combined with no-equality, eliminates all conditional moves in critical path. Only branches, moves, and arithmetic.</td><td>Can't precompute number of iterations, but can simply upper bound them.</td></tr>
<tr><td>Linear search.</td><td>Check two per cycle instead of every other cycle.</td><td>Check two per cycle instead of every other cycle.</td><td>Linear search can speculatively execute and check many elements while interpolation search can't.</td></tr>
</table>

Binary Left + Size
---------------------
<table>
<tr><th>Method</th><th>Size Impact</th></tr>
<tr><td>Reduce size to power of two</td><td>Save on the subtraction in core loop.</td></tr>
<tr><td>Precompute logarithm and use for loop.</td><td>Run-time loop unrolling.</td></tr>
<tr><td>Linear search.</td><td>Check two per cycle instead of every other cycle. Worse at first.</td></tr>
</table>

Note that linear search version is best with a lower unroll factor relative to recursive version. This makes sense since the default is unroll factor of 8, so the linear search version will never use the unrolled version.

Binary Left + Right
----------------------
<table>
<tr><th>Method</th><th>LR Impact</th></tr>
<tr><td>Overflow math</td><td>Increased ILP.</td></tr>
<tr><td>Don't test equality.</td><td>Remove 1 out of 3 conditional moves and save an increment.</td></tr>
<tr><td>Precompute logarithm and use for loop.</td><td>Run-time loop unrolling.</td></tr>
<tr><td>Use for loop without testing for equality.</td><td>Eliminates all conditional moves in critical path. Only branches, moves, and arithmetic.</td></tr>
<tr><td>Linear search.</td><td>Check two per cycle instead of every other cycle.</td></tr>
</table>

Note that the for loop LR is testing for equality.

Not testing for equality has one effect. Using a for loop has a different effect. In combination, a third effect. How to discuss?

Note that the size version already is using the overflow arithmetic, not testing for equality and for loop unrolling.

Interpolation
---------------
An interpolation is defined to be
left + (x - A[left]) / (A[right]-A[left]) * (right-left)
<table>
<tr><th>Method</th><th>Impact</th></tr>
<tr><td>Precompute part of the interpolation.</td><td>Saves several operations and type conversions in the most critical path.</td></tr>
<tr><td>Linear search + 1 interpolation</td><td>Linear search can speculatively execute and check many elements while interpolation search can't.</td></tr>
<tr><td>Lookup Table Division</td><td>Save on conversions at the expense of some accuracy, moves, and register pressure. I expect lower CPI with LUT due to instruction parallelism.</td></tr>
<tr><td>Store value in struct rather than array.</td><td>Saves a number of moves.</td></tr>
<tr><td>Interpolate from 0 to the max value.</td><td>Saves a subtraction.</td></tr>
</table>

It's important to respect your code size.
* Doing division on floating point calculation of interpolation before the multiplication was a big win. One reason is that floating point division works exactly if numerator = denominator, but loses that after you do the multiplication and lose the precision.

In [ ]:
call(['make', 'release', 'N_SAMPLES=-1', 'NSORT=1', 'N_RUNS=1000'])
sz = 1000
n_thds = 1
algs_ns = [[i, run] + list(alg_ns)
           for i, seed in enumerate(seeds)
           for run, alg_ns in read_csv(popen('./search {} {} {} {}'.format(sz, seed, n_thds, algs))).iterrows()]

In [ ]:
algs_ns_df = DataFrame.from_records(algs_ns, columns=['seed', 'run'] + algs.split()
                                   ).set_index(['seed', 'run'])
improve_ns = DataFrame([(alg, cycles.median(), quantile_p(cycles, .75),
                         quantile_p(cycles, .25))
                        for alg, cycles in algs_ns_df.iteritems()]
                       , columns=['alg', 'median', 'plus25', 'minus25']
                      ).set_index(['alg'])
for t_algs in [b_lr_algs, b_sz_algs, i_algs]:
    improve_ns['median'].loc[t_algs.split()] = (improve_ns.loc[t_algs.split()[0]
                                                              ]['median'] /
                                                improve_ns.loc[t_algs.split()]['median'])
    for quantile in ['plus25', 'minus25']:
        improve_ns[quantile].loc[t_algs.split()] = (improve_ns.loc[t_algs.split()
                                                                  ][quantile] *
                                                    improve_ns.loc[t_algs.split()
                                                                  ]['median'])

In [ ]:
#print(improve_ns.to_csv('improve.dat', index_label='alg', sep=' '))
for name, plot_algs in zip(['b-lr', 'b-sz', 'i'], [b_lr_algs, b_sz_algs, i_algs]):
    g = improve_ns.loc[plot_algs.split()]
    g.to_csv('{}-improve.dat'.format(name), index_label='alg', sep=' ')
    g['median'].plot.bar(yerr=g['plus25'])
    show()

Interpolation Performance Predictors
===================

1. Run metrics on data.
2. Combine metrics with performance data.
3. Show predictiveness of metrics.
4. Plot most predictive measurements.

Currently only looking at 1K size. Consider looking at all sizes.

In [ ]:
def read_ints(seed):
    return [int(num) for num
            in popen('./search 1000 {} 1'.format(seed)).read().split()]
def interpolate(x, n, y_l, y_r):
    if n < 1: return 0
    return floor((x-y_l) / (y_r - y_l) * (n-1))
def interpolate_l(x, l):
    return interpolate(x, len(l), l[0], l[-1])

def avg(l):
    return sum(l)/len(l)
def dist(a_l, b_l):
    return [abs(a - b) for a,b in zip(a_l, b_l)]
def r2(x,y):
    m, b, r, _, _ = linregress(x,y)
    return r**2
# TODO verify this matches definition
def smoothness(nums):
    adj_dist = [abs(d) for d in dist(nums[1:], nums[:-1])]
    return max(adj_dist) / min(adj_dist)

call(['make', 'dump'])
    
seed_metrics = []
for seed in seeds:
    nums = sorted(read_ints(seed))
    guesses = [interpolate_l(x,nums) for x in nums]
    d = sorted(dist(range(len(nums)), guesses))
    seed_metrics.append((d[-1], d[int(len(d)*.9)], avg(d),
                   r2(range(len(nums)), guesses),
                   smoothness(nums)))

#dist_d = DataFrame(dist_d).set_index('file').sort_values('l1')

In [ ]:
seed_metrics_df = DataFrame(seed_metrics,
                            columns=['max', '90', 'l1', 'r2', 'smoothness'])
seeds_ns = DataFrame({
    'seed': Series(seeds),
    'ns': algs_ns_df['i-seq'].unstack(0).median()
})
# require that there be an odd number of seeds so that median is always
# in the set.
median_nums = sorted(read_ints(
    seeds_ns[seeds_ns['ns'] == seeds_ns['ns'].median()]['seed'].iloc[0]))

cdf = Series(dist(range(len(median_nums)),
           [interpolate_l(x, median_nums) for x in median_nums]))
predict = Series([r2(series, seeds_ns['ns'])
                  for _, series in seed_metrics_df.iteritems()],
                 list(seed_metrics_df))
seed_metrics_df['ns'] = seeds_ns['ns']
ns_metrics = seed_metrics_df.sort_values(['ns']).reset_index()[['l1', 'ns']]
print(ns_metrics)

In [ ]:
print(ns_metrics)
cdf.hist(cumulative=True).plot()
cdf.to_csv('cdf.dat', sep=' ', index=False)
show()

predict.name = 'r2'
predict.index.name='metric'
predict.to_csv('predict.dat', sep=' ', header=True)
predict.plot.bar()
show()

ns_metrics.to_csv('metrics.dat', sep=' ', header=True, index_label='dataset')
ns_metrics.plot.scatter(x='ns',y='l1')
show()

Best for Size and Threads
==============

In [ ]:
szs_thds_seeds_runs_ns_m = []
for sz, n_runs in zip(szs, [1024] + [10]*8):
    call(['make', 'release', 'N_SAMPLES=-1', 'NSORT=1', 'N_RUNS={}'.format(n_runs)])
    szs_thds_seeds_runs_ns_m.append([
        [sz, n_thds, i, run] + list(alg_ns)
        for n_thds in range(1, max_n_thds+1)
        for i, seed in enumerate(seeds)
        for run, alg_ns in read_csv(popen('./search {} {} {} {}'.format(sz, seed, n_thds, sz_algs))).iterrows()
    ])
szs_thds_seeds_runs_ns_m = [x for l in szs_thds_seeds_runs_ns_m for x in l]

In [ ]:
#szs_thds_seeds_runs_ns = read_csv('szs_thds_seeds_runs_ns.csv')[
#    ['sz', 'n_thds', 'seed'] + sz_algs.split()]
szs_thds_seeds_runs_ns = DataFrame(szs_thds_seeds_runs_ns_m, columns=(['sz', 'n_thds', 'seed', 'run'] + sz_algs.split()))
szs_thds_seeds_runs_ns.to_csv('szs_thds_seeds_runs_ns.csv')

In [ ]:
szs_thds_ns_m = []
for sz, g1 in szs_thds_seeds_runs_ns.groupby(['sz']):
    ns_1 = g1[g1['n_thds'] == 1].median()[sz_algs.split()]
    for n_thds, g2 in g1.groupby(['n_thds']):
        szs_thds_ns_m.append([sz, n_thds, max([g2[alg].median() - ns_1[alg] for alg in sz_algs.split()] / ns_1)])
    #szs_thds_ns_m.append([sz, n_thds] + [g[alg].median() for alg in sz_algs.split()])
szs_thds_ns = DataFrame(szs_thds_ns_m, columns=['sz', 'n_thds', 'diff'])
szs_seeds_ns = DataFrame([[sz, seed] + [g[alg].median() for alg in sz_algs.split()]
                                        for (sz, seed), g in szs_thds_seeds_runs_ns[szs_thds_seeds_runs_ns['n_thds'] == 1
                                                                                   ].groupby(['sz', 'seed'])],
                         columns=['sz', 'seed'] + sz_algs.split()
                        ).set_index(['sz', 'seed']).stack().unstack(0).reorder_levels([1,0]).sort_index()
szs_ns = DataFrame([[sz] + [g[alg].median() for alg in sz_algs.split()]
                                        for sz, g in szs_thds_seeds_runs_ns[szs_thds_seeds_runs_ns['n_thds'] == 1
                                                                                   ].groupby(['sz'])]
                        , columns=['sz'] + sz_algs.split()
                  ).set_index(['sz'])
szs_thds_diff = szs_thds_ns[szs_thds_ns['n_thds'] > 1].set_index(['sz', 'n_thds']).stack().unstack(0) * 100
szs_thds_diff.index = szs_thds_diff.index.droplevel(1)

norm_szs_ns = matrix_norm_vec(szs_ns, szs_ns['b-sz-lin'])

In [ ]:
szs_thds_diff.plot.bar()
szs_thds_diff.to_csv('szs-thds-diff.dat', sep=' ')

szs_ns.to_csv('szs-ns.dat', sep=' ')
szs_ns.plot.bar()

norm_szs_ns.to_csv('szs-ns-norm.dat', sep=' ')
norm_szs_ns.plot.bar()

fig, ax = subplots()
for (alg, g), clr in zip(szs_seeds_ns.groupby(level=[0]), ['r', 'g', 'y', 'b']):
    g.plot.box(color=clr, ax=ax, logy=True)
    print(clr, alg)
show()